In [3]:
import xarray as xr
import numpy as np
import datetime
import glob
import os
import gsw
import pandas as pd
import matplotlib.pyplot as plt

yeart='2015'
montht='10'

#DATE REF
date_1 = datetime.datetime.strptime('01/01/1950', "%m/%d/%Y")

ROOT = '/home1/ballycotton/DATA/PIRATE/SDL_INTERP/'+yeart+'/'
path = glob.glob(os.path.join(ROOT+'*'+montht+'.nc'))
print yeart,'-',montht,': ','1-open ',path
#CUSTOM INDEX ARGO
ARGO=xr.open_dataset('argo_index_2005-2015_EDW.nc')
#OPEN
OCC=xr.open_mfdataset(path,decode_times=False)

#DELTA TIMES FOR MISSING CYCLE_NUMBER
#When interpolation was done, delta max was set to 10min but 24h seems legit...
#Should probably make a "Why give a damn on time encoding can save some (actual) time" talk someday

OCC.CYCLE_NUMBER.load()
for k in OCC.N_OBS.values:
    if np.isnan(OCC.CYCLE_NUMBER[k].values):      
        AA=ARGO.where(ARGO.wmo==int(OCC.STATION_IDENTIFIER[k].values),drop=True)  
        if len(AA.index)>0:
            dx=date_1 + datetime.timedelta(days=float(OCC.JULD[k].values))
            CC=np.array(dx.strftime('%Y-%m-%dT%H:%M:%S'),dtype='datetime64')
            delta=np.abs(CC-AA.date.values).min()
            indw=np.abs(CC-AA.date.values).argmin()
            print k,OCC.STATION_IDENTIFIER[k].values,CC,'  ##  ',indw,AA.cycle_cumber[indw].values,\
            AA.date[indw].values,np.timedelta64(delta,'m')          
            #if delta<=np.timedelta64(24,'h'):                
            #    #ASSIGN CYCLE_NUMBER
            #    OCC.CYCLE_NUMBER[k]=AA.cycle_cumber[indw].values            
        else:
            print k,OCC.STATION_IDENTIFIER[k].values,' not found in argo index'    

2015 - 10 :  1-open  ['/home1/ballycotton/DATA/PIRATE/SDL_INTERP/2015/OCCITENS-EDW-SDL-201510.nc']
12 6901121  2015-10-02T11:10:20   ##   11 138 2015-07-04T11:07:54.000000000 129602 minutes
22 6901122  2015-10-03T10:27:17   ##   4 138 2015-07-05T07:07:17.000000000 129800 minutes
50 6901417   not found in argo index
76 6901142  2015-10-08T09:01:09   ##   11 106 2015-09-08T11:09:07.000000000 43072 minutes
114 6901121  2015-10-12T11:07:37   ##   11 138 2015-07-04T11:07:54.000000000 143999 minutes
121 6901122  2015-10-13T07:06:30   ##   4 138 2015-07-05T07:07:17.000000000 143999 minutes
156 6901417   not found in argo index
168 1901218  2015-10-17T22:02:00   ##   68 151 2015-09-07T21:56:00.000000000 57606 minutes
176 6901142  2015-10-18T11:11:46   ##   11 106 2015-09-08T11:09:07.000000000 57602 minutes
215 6901121  2015-10-22T07:38:25   ##   11 138 2015-07-04T11:07:54.000000000 158190 minutes
220 6901122  2015-10-23T10:20:30   ##   4 138 2015-07-05T07:07:17.000000000 158593 minutes
257 690

In [69]:
my_bday=np.array(25129.55208333333,dtype='float64')
print 'float 64 : ',str(date_1 + datetime.timedelta(days=my_bday.item()))

my_bday=np.array(25130.55208333333,dtype='float32')
print 'float 32 : ',str(date_1 + datetime.timedelta(days=my_bday.item()))

float 64 :  2018-10-20 13:15:00
float 32 :  2018-10-21 13:15:56.250000


In [ ]:
## CASE STUDY
ks=44
indw=0

AA=ARGO.where(ARGO.wmo==int(OCC.STATION_IDENTIFIER[ks].values),drop=True)
# ERDDAP CONF & CALL
uri = (
    'http://www.ifremer.fr/erddap/tabledap/ArgoFloats.csv'
    '?time,pres,temp,psal'
    '&platform_number={}'
    '&cycle_number={}').format  
url = uri(    
    '\"'+OCC.STATION_IDENTIFIER[ks].values.item().strip()+'\"',
    int(AA.cycle_cumber[indw].values.item())
)
print "Loading data from Ifremer erddap of Argo:\n",url
df = pd.read_csv(url, parse_dates=True, skiprows=[1])

%matplotlib notebook
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.invert_yaxis()
ax1.grid(True)
ax2.grid(True)
f.suptitle(str(date_1 + datetime.timedelta(days=float(OCC.JULD[ks].values)))+' - '+str(AA.date[indw].values))
l1,=ax1.plot(OCC.TEMP_OBS[ks,:].values, gsw.p_from_z(OCC.STANDARD_LEVELS.values,OCC.LATITUDE[ks].values), 'b.',label='interp OCCIPUT')
l2,=ax1.plot(df['temp'],df['pres'],'r.',label='ERDDAP')
ax1.set_xlabel('TEMP')
ax1.set_ylabel('PRES')
ax1.legend(handles=[l1, l2])

ax2.plot(OCC.PSAL_OBS[ks,:].values, gsw.p_from_z(OCC.STANDARD_LEVELS.values,OCC.LATITUDE[ks].values), 'b.')
ax2.plot(df['psal'],df['pres'],'r.')
ax2.set_xlabel('PSAL')


In [ ]:
#SAVE NEW FILE
#OCC.to_netcdf(path[0][:-3]+'_v2.nc')